### This code generate the instruction spread sheet in Gorilla

### Import library and define function

In [22]:
import os
import glob
import random
import shutil
import openpyxl

In [23]:
def combine_rows_from_xlsx_files(source_folder, new_folder):
    os.makedirs(new_folder, exist_ok=True)

    subfolders = ['len4-rand']
    selected_rows = {'len4-rand': {'_female': [5, 10], '_male': [6,11]}}

    new_wb = openpyxl.Workbook()
    new_ws = new_wb.active

    header_written = False

    for subfolder in subfolders:
        xlsx_files = [f for f in os.listdir(os.path.join(source_folder, subfolder)) if f.endswith('.xlsx')]

        for gender in ['_female', '_male']:
            xlsx_file = next((f for f in xlsx_files if gender in f.lower()), None)
            if not xlsx_file:
                continue

            wb = openpyxl.load_workbook(os.path.join(source_folder, subfolder, xlsx_file))
            ws = wb.active

            if not header_written:
                new_ws.append([cell.value for cell in ws[1]])
                header_written = True

            selected_rows_list = selected_rows[subfolder][gender]

            # Find the "Audio_Filename" column index
            audio_filename_col_index = None
            for idx, cell in enumerate(ws[1]):
                if cell.value == "Audio_Filename":
                    audio_filename_col_index = idx + 1
                    break

            for selected_row in selected_rows_list:
                new_ws.append([cell.value for cell in ws[selected_row]])

                if audio_filename_col_index:
                    audio_filename = ws.cell(row=selected_row, column=audio_filename_col_index).value
                    audio_file_path = os.path.join(source_folder, subfolder, audio_filename)
                    if os.path.isfile(audio_file_path):
                        new_audio_filename = os.path.splitext(audio_filename)[0] + '_instruction' + os.path.splitext(audio_filename)[1]
                        new_audio_file_path = os.path.join(new_folder, new_audio_filename)
                        shutil.copy(audio_file_path, new_audio_file_path)

                        # Update the "Audio_Filename" in the new xlsx file
                        new_ws.cell(row=new_ws.max_row, column=audio_filename_col_index).value = new_audio_filename

    new_wb.save(os.path.join(new_folder, 'combined_rows.xlsx'))


In [19]:
# def combine_rows_from_xlsx_files(source_folder, new_folder):
#     os.makedirs(new_folder, exist_ok=True)

#     # subfolders = ['len6-fixed', 'len6-rand']
#     # selected_rows = {'len6-fixed': {'_female': 10, '_male': 11}, 'len6-rand': {'_female': 5, '_male': 6}}

#     subfolders = ['len4-rand']
#     selected_rows = {'len4-rand': {'_female': [5, 10], '_male': [6,11]}} 

#     new_wb = openpyxl.Workbook()
#     new_ws = new_wb.active

#     header_written = False

#     for subfolder in subfolders:
#         xlsx_files = [f for f in os.listdir(os.path.join(source_folder, subfolder)) if f.endswith('.xlsx')]

#         for gender in ['_female', '_male']:
#             #select xlsx_filw with gender in its name
#             xlsx_file = next((f for f in xlsx_files if gender in f.lower()), None)
#             if not xlsx_file:
#                 continue

#             wb = openpyxl.load_workbook(os.path.join(source_folder, subfolder, xlsx_file))
#             ws = wb.active

#             if not header_written:
#                 new_ws.append([cell.value for cell in ws[1]])
#                 header_written = True

#             selected_row = selected_rows[subfolder][gender]
#             #print(selected_row)
#             new_ws.append([cell.value for cell in ws[selected_row]])

#             # Find the "Audio_Filename" column index
#             audio_filename_col_index = None
#             for idx, cell in enumerate(ws[1]):
#                 if cell.value == "Audio_Filename":
#                     audio_filename_col_index = idx + 1
#                     break

#             if audio_filename_col_index:
#                 audio_filename = ws.cell(row=selected_row, column=audio_filename_col_index).value
#                 audio_file_path = os.path.join(source_folder, subfolder, audio_filename)
#                 if os.path.isfile(audio_file_path):
#                     new_audio_filename = os.path.splitext(audio_filename)[0] + '_instruction' + os.path.splitext(audio_filename)[1]
#                     new_audio_file_path = os.path.join(new_folder, new_audio_filename)
#                     shutil.copy(audio_file_path, new_audio_file_path)

#                     # Update the "Audio_Filename" in the new xlsx file
#                     new_ws.cell(row=new_ws.max_row, column=audio_filename_col_index).value = new_audio_filename

#     new_wb.save(os.path.join(new_folder, 'combined_rows.xlsx'))

In [24]:
def replace_rows_in_xlsx(copy_xlsx, xlsx_file1, xlsx_file2, output_folder, name):
    os.makedirs(output_folder, exist_ok=True)
    
    copy_wb = openpyxl.load_workbook(copy_xlsx)
    copy_ws = copy_wb.active

    wb1 = openpyxl.load_workbook(xlsx_file1)
    ws1 = wb1.active
    wb2 = openpyxl.load_workbook(xlsx_file2)
    ws2 = wb2.active

    non_empty_rows1 = [row for row in ws1.iter_rows(min_row=2) if any(cell.value for cell in row)]
    non_empty_rows2 = [row for row in ws2.iter_rows(min_row=2) if any(cell.value for cell in row)]

    replace_rows_copy = {4: 0, 6: 0, 8: 1, 10: 2, 12: 3, 15: 0, 17: 0, 19: 1, 21: 2, 23: 3}

    for row_num, source_row in replace_rows_copy.items():
        if row_num < 15:
            source_ws = ws1
            source_row_data = non_empty_rows1[source_row]
        else:
            source_ws = ws2
            source_row_data = non_empty_rows2[source_row]

        for idx, cell in enumerate(source_row_data):
            copy_ws.cell(row=row_num, column=idx + 1).value = cell.value

    output_file = os.path.join(output_folder, 'V4-'+name+'-task-instruction.xlsx')
    copy_wb.save(output_file)

## AP Instruction

In [27]:
### AP-Pitch
source_folder = "E:/cmu/lab project/stimuli generation/V5-syllable-4-semitone-interval-10-rand/task_type_ap/Block_2_pitch"
new_folder = "E:/cmu/lab project/stimuli generation/V5-spread_sheet/Task_AP/AP_2_section_Instruction/Pitch"
#source_folder = "E:/cmu/lab project/stimuli generation/V4-syllable-4-semitone-interval-10/task_type_ap/Block_5_pitch"
#new_folder = "E:/cmu/lab project/stimuli generation/V4-spread_sheet/Task_AP/AP_4_section_Instruction/Pitch"
if not os.path.exists(new_folder):
        os.makedirs(new_folder)

combine_rows_from_xlsx_files(source_folder, new_folder)

### AP-Word
source_folder = "E:/cmu/lab project/stimuli generation/V5-syllable-4-semitone-interval-10-rand/task_type_ap/Block_1_word"
new_folder = "E:/cmu/lab project/stimuli generation/V5-spread_sheet/Task_AP/AP_2_section_Instruction/Word"
# source_folder = "E:/cmu/lab project/stimuli generation/V4-syllable-4-semitone-interval-10/task_type_ap/Block_5_word"
# new_folder = "E:/cmu/lab project/stimuli generation/V4-spread_sheet/Task_AP/AP_4_section_Instruction/Word"
if not os.path.exists(new_folder):
        os.makedirs(new_folder)

combine_rows_from_xlsx_files(source_folder, new_folder)

In [28]:
copy_xlsx = "E:/cmu/lab project/stimuli generation/V4-spread_sheet/Task_AP/AP_4_section_Instruction/AP_instruction_template.xlsx"
xlsx_file1 = "E:/cmu/lab project/stimuli generation/V5-spread_sheet/Task_AP/AP_2_section_Instruction/Pitch/combined_rows.xlsx"
xlsx_file2 = "E:/cmu/lab project/stimuli generation/V5-spread_sheet/Task_AP/AP_2_section_Instruction/Word/combined_rows.xlsx"
output_folder = "E:/cmu/lab project/stimuli generation/V5-spread_sheet/Task_AP/AP_2_section_Instruction"
name = 'AP'
replace_rows_in_xlsx(copy_xlsx, xlsx_file1, xlsx_file2, output_folder, name)


FileNotFoundError: [Errno 2] No such file or directory: 'E:/cmu/lab project/stimuli generation/V4-spread_sheet/Task_AP/AP_4_section_Instruction/AP_instruction_template.xlsx'

## SD Instruction

In [29]:
# SD-Pitch
source_folder = "E:/cmu/lab project/stimuli generation/V5-syllable-4-semitone-interval-10-rand/task_type_sd/Block_2_pitch"
new_folder = "E:/cmu/lab project/stimuli generation/V5-spread_sheet/Task_SD/SD_2_section_Instruction/Pitch"
# source_folder = "E:/cmu/lab project/stimuli generation/V4-syllable-4-semitone-interval-10/task_type_sd/Block_5_pitch"
# new_folder = "E:/cmu/lab project/stimuli generation/V4-spread_sheet/Task_SD/SD_4_section_Instruction/Pitch"
if not os.path.exists(new_folder):
        os.makedirs(new_folder)

combine_rows_from_xlsx_files(source_folder, new_folder)

# SD-Word
# source_folder = "E:/cmu/lab project/stimuli generation/V4-syllable-4-semitone-interval-10/task_type_sd/Block_5_word"
# new_folder = "E:/cmu/lab project/stimuli generation/V4-spread_sheet/Task_SD/SD_4_section_Instruction/Word"
source_folder = "E:/cmu/lab project/stimuli generation/V5-syllable-4-semitone-interval-10-rand/task_type_sd/Block_1_word"
new_folder = "E:/cmu/lab project/stimuli generation/V5-spread_sheet/Task_SD/SD_2_section_Instruction/Word"
if not os.path.exists(new_folder):
        os.makedirs(new_folder)

combine_rows_from_xlsx_files(source_folder, new_folder)

In [28]:
copy_xlsx = "E:/cmu/lab project/stimuli generation/V4-spread_sheet/Task_SD/SD_4_section_Instruction/SD_instruction_template.xlsx"
xlsx_file1 = "E:/cmu/lab project/stimuli generation/V5-spread_sheet/Task_SD/SD_2_section_Instruction/Pitch/combined_rows.xlsx"
xlsx_file2 = "E:/cmu/lab project/stimuli generation/V5-spread_sheet/Task_SD/SD_2_section_Instruction/Word/combined_rows.xlsx"
output_folder = "E:/cmu/lab project/stimuli generation/V5-spread_sheet/Task_SD/SD_2_section_Instruction"
name = 'SD'
replace_rows_in_xlsx(copy_xlsx, xlsx_file1, xlsx_file2, output_folder,name)